In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import polars as pl
%reload_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv(
    "data/Euronext_Equities_2025-08-19.csv",
    sep=";",          # Euronext uses semicolons
    skiprows=0,       # adjust if file has metadata rows
    on_bad_lines="skip"  # skip problematic rows if any
)


In [3]:
possible_markets = ['Oslo Børs', 'Euronext Paris', 'Euronext Amsterdam', 'Euronext Brussels', 'Euronext Lisbon']
filtered_df = df[df['Market'].isin(possible_markets)]
filtered_df

,Name,ISIN,Symbol,Market,Currency,Open Price,High Price,low Price,last Price,last Trade MIC Time,Time Zone,Volume,Turnover,Closing Price,Closing Price DateTime
3,2020 BULKERS,BMG9156K1018,2020,Oslo Børs,NOK,"133,50","135,00","133,00","133,10",19/08/2025 16:25,CET,107862,"14447074,90","133,10",19/08/2025
14,74SOFTWARE,FR0011040500,74SW,Euronext Paris,EUR,"39,10","40,00","39,10","39,80",19/08/2025 17:35,CET,3538,"139815,50","39,80",19/08/2025
20,AALBERTS NV,NL0000852564,AALB,Euronext Amsterdam,EUR,"30,12","30,80","30,12","30,74",19/08/2025 17:35,CET,159908,"4903828,56","30,74",19/08/2025
24,AB INBEV,BE0974293251,ABI,Euronext Brussels,EUR,"53,20","53,58","52,90","53,42",19/08/2025 17:39,CET,954025,"50925650,92","53,42",19/08/2025
25,AB SCIENCE,FR0010557264,AB,Euronext Paris,EUR,"1,33","1,344","1,32","1,32",19/08/2025 17:35,CET,110739,"147289,34","1,32",19/08/2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4024,ZALARIS,NO0010708910,ZAL,Oslo Børs,NOK,"76,00","77,00","76,00","77,00",19/08/2025 16:25,CET,882,"67622,40","77,00",19/08/2025
4025,ZAPTEC,NO0010713936,ZAP,Oslo Børs,NOK,"26,25","26,50","25,55","25,95",19/08/2025 16:27,CET,554446,"14464193,45","25,95",19/08/2025
4027,ZCI LIMITED,BMG9887P1068,CV,Euronext Paris,EUR,"0,17","0,20","0,17","0,20",04/08/2015 16:20,CET,21826,"3755,94","0,20",19/08/2025
4029,ZELLUNA,NO0013524942,ZLNA,Oslo Børs,NOK,"13,50","13,676","13,372","13,502",19/08/2025 16:25,CET,4987,"67291,022","13,502",19/08/2025


In [4]:
market_grouped_equities = filtered_df.groupby('Market')

In [5]:
market_grouped_equities.get_group('Oslo Børs')

,Name,ISIN,Symbol,Market,Currency,Open Price,High Price,low Price,last Price,last Trade MIC Time,Time Zone,Volume,Turnover,Closing Price,Closing Price DateTime
3,2020 BULKERS,BMG9156K1018,2020,Oslo Børs,NOK,"133,50","135,00","133,00","133,10",19/08/2025 16:25,CET,107862,"14447074,90","133,10",19/08/2025
39,ABG SUNDAL COLLIER,NO0003021909,ABG,Oslo Børs,NOK,"6,96","7,05","6,95","6,95",19/08/2025 16:25,CET,2305357,"17000246,86","6,95",19/08/2025
44,ABL GROUP,NO0010715394,ABL,Oslo Børs,NOK,"9,36","9,36","9,14","9,14",19/08/2025 16:15,CET,47463,"437236,10","9,14",19/08/2025
116,AF GRUPPEN,NO0003078107,AFG,Oslo Børs,NOK,"160,20","163,60","160,00","163,40",19/08/2025 16:25,CET,29299,"4764223,60","163,40",19/08/2025
134,AGILYX,NO0010872468,AGLX,Oslo Børs,NOK,"24,50","25,00","24,50","25,00",19/08/2025 16:25,CET,10652,"264156,40","25,00",19/08/2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3987,WILH. WILHELMSEN B,NO0010576010,WWIB,Oslo Børs,NOK,"470,00","475,00","470,00","470,50",19/08/2025 16:27,CET,6414,"3026836,00","470,50",19/08/2025
4016,YARA INTERNATIONAL,NO0010208051,YAR,Oslo Børs,NOK,"376,60","378,80","374,50","375,60",19/08/2025 16:26,CET,495690,"186290912,288","375,60",19/08/2025
4024,ZALARIS,NO0010708910,ZAL,Oslo Børs,NOK,"76,00","77,00","76,00","77,00",19/08/2025 16:25,CET,882,"67622,40","77,00",19/08/2025
4025,ZAPTEC,NO0010713936,ZAP,Oslo Børs,NOK,"26,25","26,50","25,55","25,95",19/08/2025 16:27,CET,554446,"14464193,45","25,95",19/08/2025


In [6]:
df = df.dropna()
oslo_stocks = df.loc[df['Market'] == 'Oslo Børs']

In [7]:
oslo_stocks

,Name,ISIN,Symbol,Market,Currency,Open Price,High Price,low Price,last Price,last Trade MIC Time,Time Zone,Volume,Turnover,Closing Price,Closing Price DateTime
3,2020 BULKERS,BMG9156K1018,2020,Oslo Børs,NOK,"133,50","135,00","133,00","133,10",19/08/2025 16:25,CET,107862,"14447074,90","133,10",19/08/2025
39,ABG SUNDAL COLLIER,NO0003021909,ABG,Oslo Børs,NOK,"6,96","7,05","6,95","6,95",19/08/2025 16:25,CET,2305357,"17000246,86","6,95",19/08/2025
44,ABL GROUP,NO0010715394,ABL,Oslo Børs,NOK,"9,36","9,36","9,14","9,14",19/08/2025 16:15,CET,47463,"437236,10","9,14",19/08/2025
116,AF GRUPPEN,NO0003078107,AFG,Oslo Børs,NOK,"160,20","163,60","160,00","163,40",19/08/2025 16:25,CET,29299,"4764223,60","163,40",19/08/2025
134,AGILYX,NO0010872468,AGLX,Oslo Børs,NOK,"24,50","25,00","24,50","25,00",19/08/2025 16:25,CET,10652,"264156,40","25,00",19/08/2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3987,WILH. WILHELMSEN B,NO0010576010,WWIB,Oslo Børs,NOK,"470,00","475,00","470,00","470,50",19/08/2025 16:27,CET,6414,"3026836,00","470,50",19/08/2025
4016,YARA INTERNATIONAL,NO0010208051,YAR,Oslo Børs,NOK,"376,60","378,80","374,50","375,60",19/08/2025 16:26,CET,495690,"186290912,288","375,60",19/08/2025
4024,ZALARIS,NO0010708910,ZAL,Oslo Børs,NOK,"76,00","77,00","76,00","77,00",19/08/2025 16:25,CET,882,"67622,40","77,00",19/08/2025
4025,ZAPTEC,NO0010713936,ZAP,Oslo Børs,NOK,"26,25","26,50","25,55","25,95",19/08/2025 16:27,CET,554446,"14464193,45","25,95",19/08/2025


In [8]:
oslo_stocks
oslo_tickers = list(oslo_stocks['Symbol'].values)
oslo_tickers = ['OSEBX.OL'] + ['.'.join([i, 'OL']) for i in oslo_tickers]
ticker_data = []

In [9]:
oslo_tickers

['OSEBX.OL',
 '2020.OL',
 'ABG.OL',
 'ABL.OL',
 'AFG.OL',
 'AGLX.OL',
 'AIRX.OL',
 'AKAST.OL',
 'AKER.OL',
 'AKBM.OL',
 'AKRBP.OL',
 'ACC.OL',
 'AKH.OL',
 'AKSO.OL',
 'AKVA.OL',
 'AMSC.OL',
 'ARCH.OL',
 'AZT.OL',
 'AFK.OL',
 'ARR.OL',
 'ATEA.OL',
 'ASAS.OL',
 'ASA.OL',
 'AURG.OL',
 'AUSS.OL',
 'AUTO.OL',
 'AGAS.OL',
 'ACR.OL',
 'B2I.OL',
 'BAKKA.OL',
 'BGBIO.OL',
 'BEWI.OL',
 'BIEN.OL',
 'BNOR.OL',
 'BONHR.OL',
 'BOR.OL',
 'BRG.OL',
 'BOUV.OL',
 'BWE.OL',
 'BWLPG.OL',
 'BWO.OL',
 'BMA.OL',
 'CADLR.OL',
 'CAPSL.OL',
 'CAVEN.OL',
 'CRNA.OL',
 'CLOUD.OL',
 'CMBTO.OL',
 'CONTX.OL',
 'DNB.OL',
 'DNO.OL',
 'DOFG.OL',
 'EIOF.OL',
 'EMGS.OL',
 'ELK.OL',
 'ELABS.OL',
 'ELMRA.OL',
 'ELO.OL',
 'ENDUR.OL',
 'ENSU.OL',
 'ENTRA.OL',
 'ENVIP.OL',
 'EQNR.OL',
 'EQVA.OL',
 'EPR.OL',
 'FLNG.OL',
 'FRO.OL',
 'GENT.OL',
 'GJF.OL',
 'GOGL.OL',
 'GOD.OL',
 'GSF.OL',
 'GYL.OL',
 'HAFNI.OL',
 'HGSB.OL',
 'HAVI.OL',
 'HERMA.OL',
 'HEX.OL',
 'HPUR.OL',
 'HSHP.OL',
 'HBC.OL',
 'HYPRO.OL',
 'HAUTO.OL',
 'HSPG.OL'

In [10]:
j = yf.Ticker(oslo_tickers[0])

In [11]:
j.history(start="2010-01-01", end="2024-01-01")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2013-03-05 00:00:00+01:00,472.570007,479.269989,472.570007,479.040009,0,0.0,0.0
2013-03-06 00:00:00+01:00,479.019989,480.130005,477.070007,477.089996,0,0.0,0.0
2013-03-07 00:00:00+01:00,477.089996,479.299988,475.880005,475.880005,0,0.0,0.0
2013-03-08 00:00:00+01:00,475.850006,480.649994,475.850006,478.940002,0,0.0,0.0
2013-03-11 00:00:00+01:00,478.959991,479.369995,476.790009,479.369995,0,0.0,0.0
...,...,...,...,...,...,...,...
2023-12-21 00:00:00+01:00,1296.849976,1296.900024,1287.199951,1292.589966,75747000,0.0,0.0
2023-12-22 00:00:00+01:00,1292.560059,1298.410034,1291.949951,1295.420044,69655900,0.0,0.0
2023-12-27 00:00:00+01:00,1294.969971,1310.420044,1294.969971,1307.099976,64415400,0.0,0.0


In [12]:
oslo_ticker_classes =  [yf.Ticker(f"{tick}") for tick in oslo_tickers]
oslo_timeseries = [tick.history(start="2021-07-01", end="2025-06-01") for tick in oslo_ticker_classes]
osl_time_dict = zip(oslo_ticker_classes, oslo_timeseries)

$ASAS.OL: possibly delisted; no price data found  (1d 2021-07-01 -> 2025-06-01)
$CMBTO.OL: possibly delisted; no price data found  (1d 2021-07-01 -> 2025-06-01) (Yahoo error = "Data doesn't exist for startDate = 1625090400, endDate = 1748728800")
$ENH.OL: possibly delisted; no price data found  (1d 2021-07-01 -> 2025-06-01) (Yahoo error = "Data doesn't exist for startDate = 1625090400, endDate = 1748728800")
$SNTIA.OL: possibly delisted; no price data found  (1d 2021-07-01 -> 2025-06-01) (Yahoo error = "Data doesn't exist for startDate = 1625090400, endDate = 1748728800")
$SWON.OL: possibly delisted; no price data found  (1d 2021-07-01 -> 2025-06-01) (Yahoo error = "Data doesn't exist for startDate = 1625090400, endDate = 1748728800")


In [13]:
a = yf.Ticker("BWO.OL").history(start="2021-07-01", end="2025-06-01", auto_adjust = False).asfreq('B').ffill()

In [14]:
a['Adj Close'].pct_change(30)

Date
2021-07-01 00:00:00+02:00         NaN
2021-07-02 00:00:00+02:00         NaN
2021-07-05 00:00:00+02:00         NaN
2021-07-06 00:00:00+02:00         NaN
2021-07-07 00:00:00+02:00         NaN
                               ...   
2025-05-26 00:00:00+02:00    0.166667
2025-05-27 00:00:00+02:00    0.173752
2025-05-28 00:00:00+02:00    0.195212
2025-05-29 00:00:00+02:00    0.195212
2025-05-30 00:00:00+02:00    0.237569
Freq: B, Name: Adj Close, Length: 1022, dtype: float64

In [15]:
osl_time_dict = dict(zip(oslo_tickers, oslo_timeseries))

In [16]:
list(osl_time_dict.keys())

['OSEBX.OL',
 '2020.OL',
 'ABG.OL',
 'ABL.OL',
 'AFG.OL',
 'AGLX.OL',
 'AIRX.OL',
 'AKAST.OL',
 'AKER.OL',
 'AKBM.OL',
 'AKRBP.OL',
 'ACC.OL',
 'AKH.OL',
 'AKSO.OL',
 'AKVA.OL',
 'AMSC.OL',
 'ARCH.OL',
 'AZT.OL',
 'AFK.OL',
 'ARR.OL',
 'ATEA.OL',
 'ASAS.OL',
 'ASA.OL',
 'AURG.OL',
 'AUSS.OL',
 'AUTO.OL',
 'AGAS.OL',
 'ACR.OL',
 'B2I.OL',
 'BAKKA.OL',
 'BGBIO.OL',
 'BEWI.OL',
 'BIEN.OL',
 'BNOR.OL',
 'BONHR.OL',
 'BOR.OL',
 'BRG.OL',
 'BOUV.OL',
 'BWE.OL',
 'BWLPG.OL',
 'BWO.OL',
 'BMA.OL',
 'CADLR.OL',
 'CAPSL.OL',
 'CAVEN.OL',
 'CRNA.OL',
 'CLOUD.OL',
 'CMBTO.OL',
 'CONTX.OL',
 'DNB.OL',
 'DNO.OL',
 'DOFG.OL',
 'EIOF.OL',
 'EMGS.OL',
 'ELK.OL',
 'ELABS.OL',
 'ELMRA.OL',
 'ELO.OL',
 'ENDUR.OL',
 'ENSU.OL',
 'ENTRA.OL',
 'ENVIP.OL',
 'EQNR.OL',
 'EQVA.OL',
 'EPR.OL',
 'FLNG.OL',
 'FRO.OL',
 'GENT.OL',
 'GJF.OL',
 'GOGL.OL',
 'GOD.OL',
 'GSF.OL',
 'GYL.OL',
 'HAFNI.OL',
 'HGSB.OL',
 'HAVI.OL',
 'HERMA.OL',
 'HEX.OL',
 'HPUR.OL',
 'HSHP.OL',
 'HBC.OL',
 'HYPRO.OL',
 'HAUTO.OL',
 'HSPG.OL'

In [17]:
earliest_date = datetime.strptime("2021-07-01", "%Y-%m-%d")
latest_date = datetime.strptime("2025-05-30", "%Y-%m-%d")
filtered_timeseries = [(ticker, osl_time_dict[ticker]) for ticker in osl_time_dict.keys() if not osl_time_dict[ticker].empty]
filtered_timeseries = [(ticker, timeseries.resample('1d').asfreq().interpolate(method='linear')) for ticker, timeseries in filtered_timeseries if ((timeseries.index[0] == earliest_date) and (timeseries.index[-1] == latest_date))]

In [18]:
osl_time_dict['CADLR.OL']

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-07-01 00:00:00+02:00,34.750000,36.169998,34.360001,36.000000,657684,0.0,0.0
2021-07-02 00:00:00+02:00,36.439999,36.794998,36.000000,36.590000,719343,0.0,0.0
2021-07-05 00:00:00+02:00,36.799999,36.799999,35.505001,35.610001,192391,0.0,0.0
2021-07-06 00:00:00+02:00,35.500000,35.685001,34.674999,34.799999,187738,0.0,0.0
2021-07-07 00:00:00+02:00,35.099998,35.430000,33.759998,34.200001,216347,0.0,0.0
...,...,...,...,...,...,...,...
2025-05-23 00:00:00+02:00,48.200001,50.250000,47.779999,50.000000,547977,0.0,0.0
2025-05-26 00:00:00+02:00,49.000000,49.560001,48.139999,48.320000,717578,0.0,0.0
2025-05-27 00:00:00+02:00,47.840000,49.799999,47.459999,49.619999,686865,0.0,0.0


In [19]:
tickers = [ts_tuple[0] for ts_tuple in filtered_timeseries]
averaged_ts = np.concatenate([(ts_tuple[1]['Close'].values + ts_tuple[1]['High'].values)/2 for ts_tuple in filtered_timeseries], axis = 1)

ValueError: need at least one array to concatenate

In [ ]:
filtered_timeseries[0]

('OSEBX.OL',
 Price             Close         High          Low         Open       Volume
 Ticker         OSEBX.OL     OSEBX.OL     OSEBX.OL     OSEBX.OL     OSEBX.OL
 Date                                                                       
 2021-07-01  1131.510010  1133.449951  1120.810059  1120.959961   81121200.0
 2021-07-02  1137.060059  1139.270020  1129.420044  1131.520020  114783100.0
 2021-07-03  1138.390045  1141.145020  1131.852539  1134.412506  106320000.0
 2021-07-04  1139.720032  1143.020020  1134.285034  1137.304993   97856900.0
 2021-07-05  1141.050018  1144.895020  1136.717529  1140.197479   89393800.0
 ...                 ...          ...          ...          ...          ...
 2025-05-26  1564.869995  1567.719971  1549.719971  1549.719971   52737300.0
 2025-05-27  1574.520020  1577.689941  1563.390015  1565.079956   63567800.0
 2025-05-28  1564.949951  1582.189941  1564.530029  1575.319946   52159900.0
 2025-05-29  1563.279968  1578.104980  1562.590027  1570.429993

In [ ]:
filtered_timeseries[1]

('2020.OL',
 Price            Close        High         Low        Open    Volume
 Ticker         2020.OL     2020.OL     2020.OL     2020.OL   2020.OL
 Date                                                                
 2021-07-01  106.102219  108.460046  106.102219  108.460046   72006.0
 2021-07-02  104.215942  106.102204  104.215942  106.102204   49503.0
 2021-07-03  104.215942  105.787827  103.430000  105.787827   60108.0
 2021-07-04  104.215942  105.473450  102.644058  105.473450   70713.0
 2021-07-05  104.215942  105.159073  101.858116  105.159073   81318.0
 ...                ...         ...         ...         ...       ...
 2025-05-26  118.812912  120.308041  117.816159  117.816159   40012.0
 2025-05-27  117.716492  119.709997  117.716492  118.812918   53790.0
 2025-05-28  117.716492  120.607076  117.417470  119.610323   62156.0
 2025-05-29  119.111942  121.952686  117.766328  118.862754   96206.5
 2025-05-30  120.507393  123.298296  118.115185  118.115185  130257.0
 
 [1430

In [ ]:
final_df = pd.DataFrame(averaged_ts, columns=tickers, index = filtered_timeseries[0][1].index)
final_df

,OSEBX.OL,2020.OL,ABG.OL,ABL.OL,AFG.OL,AGLX.OL,AIRX.OL,AKAST.OL,AKER.OL,AKBM.OL,...,VVL.OL,VOW.OL,WAWI.OL,WSTEP.OL,WWI.OL,WWIB.OL,YAR.OL,ZAL.OL,ZAP.OL,ZLNA.OL
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-01,1132.479980,107.281132,6.712875,7.573390,157.172450,30.2500,8.725000,6.535000,557.269017,73.449997,...,156.917837,40.740000,24.602812,22.413368,161.273331,156.241773,356.162452,57.999274,46.180,733.000
2021-07-02,1138.165039,105.159073,6.686289,7.540283,157.421036,29.5000,8.570000,6.410000,566.168419,72.650002,...,156.050888,39.670000,24.306968,21.907511,160.396835,155.805952,358.059599,59.174933,46.230,738.000
2021-07-03,1139.767532,105.001885,6.692935,7.540283,157.089626,29.6000,8.761666,6.393333,569.134874,72.433334,...,156.050880,40.346666,24.200358,21.933453,159.447301,155.878584,359.621220,59.109619,45.770,742.000
2021-07-04,1141.370026,104.844696,6.699582,7.540283,156.758217,29.7000,8.953333,6.376667,572.101329,72.216667,...,156.050872,41.023333,24.093749,21.959395,158.497767,155.951217,361.182840,59.044304,45.310,746.000
2021-07-05,1142.972519,104.687508,6.706228,7.540283,156.426807,29.8000,9.145000,6.360000,575.067784,72.000000,...,156.050863,41.699999,23.987139,21.985337,157.548233,156.023849,362.744460,58.978990,44.850,750.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-26,1566.294983,119.560476,6.640000,10.125000,146.699997,28.2750,1.250000,12.140000,601.000000,59.799999,...,366.000000,2.820000,81.900002,25.600000,421.000000,402.250000,373.334741,81.399998,20.825,13.649
2025-05-27,1576.104980,118.713244,6.650000,10.200000,148.000000,27.9750,1.300000,12.080000,601.000000,59.549999,...,364.000000,2.675000,81.099998,25.400001,416.250000,398.000000,374.272125,80.599998,20.325,13.335
2025-05-28,1573.569946,119.161784,6.675000,10.100000,149.599998,28.3000,1.200000,12.120000,602.000000,59.950001,...,362.000000,2.175000,80.450001,24.900000,409.500000,392.250000,373.976140,80.699997,20.150,13.396


In [ ]:
# Filter by volatility
# Filter by market cap

In [ ]:
from utils.volatility_calculator import VolatilityCalculator

In [ ]:
VolCalc = VolatilityCalculator()

In [ ]:
final_df.columns

Index(['OSEBX.OL', '2020.OL', 'ABG.OL', 'ABL.OL', 'AFG.OL', 'AGLX.OL',
       'AIRX.OL', 'AKAST.OL', 'AKER.OL', 'AKBM.OL',
       ...
       'VVL.OL', 'VOW.OL', 'WAWI.OL', 'WSTEP.OL', 'WWI.OL', 'WWIB.OL',
       'YAR.OL', 'ZAL.OL', 'ZAP.OL', 'ZLNA.OL'],
      dtype='object', length=182)

In [ ]:
import jax.numpy as jnp
VolCalc.calculate_beta(jnp.array(final_df['OSEBX.OL'].values),
                       jnp.array(final_df['OSEBX.OL'].values), 
                       30)

Array(1.0007155, dtype=float32)

In [ ]:
import polars as pl
k = VolCalc.calculate_std_log_returns_for_df(final_df, 7, 'daily')
v = VolCalc.calculate_beta_for_df(final_df,final_df[final_df.columns[0]], 30)